# Whisper's transcription plus Pyannote's Diarization 

Andrej Karpathy [suggested](https://twitter.com/karpathy/status/1574476200801538048?s=20&t=s5IMMXOYjBI6-91dib6w8g) training a classifier on top of  OpenAI [Whisper](https://openai.com/blog/whisper/) model features to identify the speaker, so we can visualize the speaker in the transcript. But, as [pointed out](https://twitter.com/tarantulae/status/1574493613362388992?s=20&t=s5IMMXOYjBI6-91dib6w8g) by Christian Perone, it seems that features from whisper wouldn't be that great for speaker recognition as its main objective is basically to ignore speaker differences.

In the following, I use [**`pyannote-audio`**](https://github.com/pyannote/pyannote-audio), a speaker diarization toolkit by Hervé Bredin, to identify the speakers, and then match it with the transcriptions of Whispr. I try it on a part of an [interview](https://youtu.be/NSp2fEQ6wyA) with Freeman Dyson. Check the result [**here**](https://majdoddin.github.io/dyson.html).

To make it easier to match the transcriptions to diarizations by speaker change, Sarah Kaiser [suggested](https://github.com/openai/whisper/discussions/264#discussioncomment-3825375) runnnig the pyannote.audio first and  then just running whisper on the split-by-speaker chunks. 
For sake of performance (and transcription quality?), we attach the audio segements into a single audio file with a silent spacer as a seperator, and run whisper on it. Enjoy it!

(For sake of performance , I also tried attaching the audio segements into a single audio file with a silent -or beep- spacer as a seperator, and run whisper on it see it on [colab](https://colab.research.google.com/drive/1HuvcY4tkTHPDzcwyVH77LCh_m8tP-Qet?usp=sharing). It [works](https://majdoddin.github.io/lexicap.html) on some audio, and fails on some (Dyson's Interview). The problem is, whisper does not reliably make a timestap on a spacer. See the discussions [#139](https://github.com/openai/whisper/discussions/139) and [#29](https://github.com/openai/whisper/discussions/29))

# Preparing the audio file

 Installing [`yt-dlp`](https://github.com/yt-dlp/yt-dlp). and downloading the [video](https://) from youtube.

In [ ]:
!pip install -U yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 4.3 MB/s 
     |████████████████████████████████| 106 kB 86.8 MB/s 
     |████████████████████████████████| 357 kB 79.4 MB/s 
     |████████████████████████████████| 193 kB 82.4 MB/s 
     |████████████████████████████████| 2.3 MB 63.1 MB/s 


Custom build of `ffmpeg` as [recommended](https://github.com/yt-dlp/yt-dlp#strongly-recommended) by `yt-dlp`.

In [10]:
!wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
video_id = 'NSp2fEQ6wyA'
video_url = 'https://youtu.be/' + video_id

In [ ]:
from yt_dlp import YoutubeDL

with YoutubeDL() as ydl: 
  info_dict = ydl.extract_info(video_url, download=False)
  video_title = info_dict.get('title', None)
  print("Title: " + video_title) # <= Here, you got the video title


[youtube] NSp2fEQ6wyA: Downloading webpage
[youtube] NSp2fEQ6wyA: Downloading android player API JSON
Title: Freeman Dyson - Pure mathematics at Cambridge: the influence of Besicovitch (23/157)


Downloading the audio from YouTube.

In [8]:
!yt-dlp -xv --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin --audio-format wav  -o youtube.wav -- {video_url}

'yt-dlp' is not recognized as an internal or external command,
operable program or batch file.


`pyannote.audio` seems to miss the first 0.5 seconds of the audio, and, therefore, we prepend a spcacer.

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from pydub import AudioSegment

spacermilli = 2000
spacer = AudioSegment.silent(duration=spacermilli)


audio = AudioSegment.from_mp3('audio.mp3') #lecun1.wav

audio = spacer.append(audio, crossfade=0)

audio.export('audio.wav', format='wav')

FileNotFoundError: [Errno 2] No such file or directory: 'audio.mp3'

# Pyannote's Diarization

[`pyannote.audio`](https://github.com/pyannote/pyannote-audio) is an open-source toolkit written in Python for **speaker diarization**. 

Based on [`PyTorch`](https://pytorch.org) machine learning framework, it provides a set of trainable end-to-end neural building blocks that can be combined and jointly optimized to build speaker diarization pipelines. 

`pyannote.audio` also comes with pretrained [models](https://huggingface.co/models?other=pyannote-audio-model) and [pipelines](https://huggingface.co/models?other=pyannote-audio-pipeline) covering a wide range of domains for voice activity detection, speaker segmentation, overlapped speech detection, speaker embedding reaching state-of-the-art performance for most of them. 

Installing `pyannote.audio`.

In [ ]:
!pip install   pyannote.audio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 4.1 MB/s 
     |████████████████████████████████| 51 kB 169 kB/s 
     |████████████████████████████████| 47 kB 5.3 MB/s 
     |████████████████████████████████| 585 kB 8.2 MB/s 
     |████████████████████████████████| 41 kB 464 kB/s 
     |████████████████████████████████| 111 kB 94.0 MB/s 
     |████████████████████████████████| 60 kB 6.9 MB/s 
     |████████████████████████████████| 79 kB 9.3 MB/s 
     |████████████████████████████████| 529 kB 45.4 MB/s 
     |████████████████████████████████| 217 kB 60.7 MB/s 
     |████████████████████████████████| 163 kB 71.1 MB/s 
     |████████████████████████████████| 498 kB 68.6 MB/s 
     |████████████████████████████████| 117 kB 92.7 MB/s 
     |████████████████████████████████| 130 kB 69.7 MB/s 
     |████████████████████████████████| 348 kB 64.9 MB/s 
     |████████████████████████████████| 209

**Important:** To load the speaker diarization pipeline, 

* accept the user conditions on both [hf.co/pyannote/speaker-diarization](https://hf.co/pyannote/speaker-diarization) and [hf.co/pyannote/segmentation](https://huggingface.co/pyannote/segmentation).
* login using `notebook_login` below

In [1]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\geekc\.cache\huggingface\token
Login successful


In [3]:
from pyannote.audio import Pipeline
import numpy as np

pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token=True)

Running pyannote.audio to generate the diarizations.

In [4]:
DEMO_FILE = {'uri': 'blabla', 'audio': 'audio.wav'}
dz = pipeline(DEMO_FILE)  

with open("diarization.txt", "w") as text_file:
    text_file.write(str(dz))

In [5]:
print(*list(dz.itertracks(yield_label = True))[:10], sep="\n")

(<Segment(9.12094, 10.0828)>, 'N', 'SPEAKER_01')
(<Segment(11.0447, 12.2934)>, 'A', 'SPEAKER_00')
(<Segment(13.4578, 19.4484)>, 'B', 'SPEAKER_00')
(<Segment(18.5034, 19.4822)>, 'O', 'SPEAKER_01')
(<Segment(20.9503, 22.9922)>, 'P', 'SPEAKER_01')
(<Segment(24.8484, 26.9916)>, 'C', 'SPEAKER_00')
(<Segment(27.7678, 31.0584)>, 'D', 'SPEAKER_00')
(<Segment(31.9022, 42.6347)>, 'E', 'SPEAKER_00')
(<Segment(44.9634, 47.3091)>, 'F', 'SPEAKER_00')
(<Segment(48.8447, 50.0766)>, 'G', 'SPEAKER_00')


# Preparing audio files according to the diarization

In [6]:
def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s

Grouping the diarization segments according to the speaker.

In [7]:
import re
dzs = open('diarization.txt').read().splitlines()

groups = []
g = []
lastend = 0

for d in dzs:   
  if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
    groups.append(g)
    g = []
  
  g.append(d)
  
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
  end = millisec(end)
  if (lastend > end):       #segment engulfed by a previous segment
    groups.append(g)
    g = [] 
  else:
    lastend = end
if g:
  groups.append(g)
print(*groups, sep='\n')

['[ 00:00:09.120 -->  00:00:10.082] N SPEAKER_01']
['[ 00:00:11.044 -->  00:00:12.293] A SPEAKER_00', '[ 00:00:13.457 -->  00:00:19.448] B SPEAKER_00']
['[ 00:00:18.503 -->  00:00:19.482] O SPEAKER_01', '[ 00:00:20.950 -->  00:00:22.992] P SPEAKER_01']
['[ 00:00:24.848 -->  00:00:26.991] C SPEAKER_00', '[ 00:00:27.767 -->  00:00:31.058] D SPEAKER_00', '[ 00:00:31.902 -->  00:00:42.634] E SPEAKER_00', '[ 00:00:44.963 -->  00:00:47.309] F SPEAKER_00', '[ 00:00:48.844 -->  00:00:50.076] G SPEAKER_00']
['[ 00:00:53.451 -->  00:00:56.691] Q SPEAKER_01', '[ 00:00:57.417 -->  00:01:01.838] R SPEAKER_01', '[ 00:01:04.791 -->  00:01:08.504] S SPEAKER_01']
['[ 00:01:08.504 -->  00:01:08.706] H SPEAKER_00', '[ 00:01:10.225 -->  00:01:14.764] I SPEAKER_00', '[ 00:01:16.502 -->  00:01:17.582] J SPEAKER_00']
['[ 00:01:20.265 -->  00:01:22.071] T SPEAKER_01']
['[ 00:01:23.117 -->  00:01:32.112] K SPEAKER_00', '[ 00:01:32.736 -->  00:01:32.871] L SPEAKER_00']
['[ 00:01:34.710 -->  00:01:34.997] U SPEA

Save the audio part corresponding to each diarization group.

In [10]:
audio = AudioSegment.from_wav("audio.wav")
gidx = -1
for g in groups:
  start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
  start = millisec(start) #- spacermilli
  end = millisec(end)  #- spacermilli
  print(start, end)
  gidx += 1
  audio[start:end].export(str(gidx) + '.wav', format='wav')

9120 10082
11044 19448
18503 22992
24848 50076
53451 68504
68504 77582
80265 82071
83117 92871
94710 94997
100448 108025
108987 122132


Freeing up some memory

In [ ]:
#del   DEMO_FILE, pipeline, spacer,  audio, dz, newAudio

# Whisper's Transcriptions

Installing Open AI whisper.

**Important:** There is a version conflict with pyannote.audio resulting in an error (see this [RP](https://github.com/pyannote/pyannote-audio/pull/1098)). Our workaround is to first run Pyannote and then whisper. You can safely ignore the error.

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cigol6gh
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-cigol6gh
     |████████████████████████████████| 5.5 MB 4.1 MB/s 
     |████████████████████████████████| 7.6 MB 49.8 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175217 sha256=bb2e6ad623cec90b71dc01bcbc77da9fed64007bcc2089e6432a02cf244989a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-a4w48hko/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


Run whisper on all audio files. Whisper generates the transcription and writes it to a file.

In [11]:
for i in range(gidx+1):
  !whisper {str(i) + '.wav'} --language en --model large

[00:00.000 --> 00:02.000]  Give me the child.


C:\Users\geekc\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:06.320]  Sara, beware. I have been generous up until now, but I can be cruel.
[00:07.120 --> 00:08.160]  Generous?


C:\Users\geekc\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:04.000]  What have you done that's generous?


C:\Users\geekc\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


^C


# Generating the HTML file from the Transcriptions and the Diarization

Reading the transcription file.

In [ ]:
!pip install -U webvtt-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Change or add to the speaker names and collors bellow as you wish `(speaker, textbox color, speaker color)`.

In [ ]:
speakers = {'SPEAKER_00':('Dyson', 'white', 'darkorange'), 'SPEAKER_01':('Interviewer', '#e1ffc7', 'darkgreen') }
def_boxclr = 'white'
def_spkrclr = 'orange'

In the generated HTML,  the transcriptions for each diarization group are written in a box, with the speaker name on the top. By clicking a transcription, the embedded video jumps to the right time .

In [ ]:
preS = '<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta http-equiv="X-UA-Compatible" content="ie=edge">\n    <title>' + \
      video_title + \
      '</title>\n    <style>\n        body {\n            font-family: sans-serif;\n            font-size: 18px;\n            color: #111;\n            padding: 0 0 1em 0;\n\t        background-color: #efe7dd;\n        }\n        table {\n             border-spacing: 10px;\n        }\n        th { text-align: left;}\n        .lt {\n          color: inherit;\n          text-decoration: inherit;\n        }\n        .l {\n          color: #050;\n        }\n        .s {\n            display: inline-block;\n        }\n        .c {\n            display: inline-block;\n        }\n        .e {\n            /*background-color: white; Changing background color */\n            border-radius: 20px; /* Making border radius */\n            width: fit-content; /* Making auto-sizable width */\n            height: fit-content; /* Making auto-sizable height */\n            padding: 5px 30px 5px 30px; /* Making space around letters */\n            font-size: 18px; /* Changing font size */\n            display: flex;\n            flex-direction: column;\n            margin-bottom: 10px;\n            white-space: nowrap;\n        }\n\n        .t {\n            display: inline-block;\n        }\n        #player {\n            position: sticky;\n            top: 20px;\n            float: right;\n        }\n    </style>\n\t<script>\n      var tag = document.createElement(\'script\');\n      tag.src = "https://www.youtube.com/iframe_api";\n      var firstScriptTag = document.getElementsByTagName(\'script\')[0];\n      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);\n      var player;\n      function onYouTubeIframeAPIReady() {\n        player = new YT.Player(\'player\', {\n          //height: \'210\',\n          //width: \'340\',\n          videoId: \'' + \
      video_id + \
      '\',\n        });\n      }\n      function jumptoTime(timepoint, id) {\n        event.preventDefault();\n        history.pushState(null, null, "#"+id);\n        player.seekTo(timepoint);\n        player.playVideo();\n      }\n    </script>\n  </head>\n  <body>\n    <h2>' + \
      video_title + \
      '</h2>\n  <i>Click on a part of the transcription, to jump to its video, and get an anchor to it in the address bar<br><br></i>\n<div  id="player"></div>\n'
postS = '\t</body>\n</html>'

In [ ]:
import webvtt

from datetime import timedelta

html = list(preS)
gidx = -1
for g in groups:  
  shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  shift = millisec(shift) - spacermilli #the start time in the original video
  shift=max(shift, 0)
  
  gidx += 1
  captions = [[(int)(millisec(caption.start)), (int)(millisec(caption.end)),  caption.text] for caption in webvtt.read(str(gidx) + '.wav.vtt')]

  if captions:
    speaker = g[0].split()[-1]
    boxclr = def_boxclr
    spkrclr = def_spkrclr
    if speaker in speakers:
      speaker, boxclr, spkrclr = speakers[speaker] 
    html.append(f'<div class="e" style="background-color: {boxclr}">\n');
    html.append(f'<span style="color: {spkrclr}">{speaker}</span><br>\n')
      
    for c in captions:
      start = shift + c[0] 

      start = start / 1000.0   #time resolution ot youtube is Second.
      startStr = '{0:02d}:{1:02d}:{2:02.2f}'.format((int)(start // 3600), 
                                              (int)(start % 3600 // 60), 
                                              start % 60)      
      #html.append(f'<div class="c">')
      #html.append(f'\t\t\t\t<a class="l" href="#{startStr}" id="{startStr}">#</a> \n')
      html.append(f'\t\t\t\t<a href="#{startStr}" id="{startStr}" class="lt" onclick="jumptoTime({int(start)}, this.id)">{c[2]}</a>\n')
      #html.append(f'\t\t\t\t<div class="t"> {c[2]}</div><br>\n')
      #html.append(f'</div>')

    html.append(f'</div>\n');

html.append(postS)
s = "".join(html)

with open("capspeaker.html", "w") as text_file:
    text_file.write(s)
print(s)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Freeman Dyson - Pure mathematics at Cambridge: the influence of Besicovitch (23/157)</title>
    <style>
        body {
            font-family: sans-serif;
            font-size: 18px;
            color: #111;
            padding: 0 0 1em 0;
	        background-color: #efe7dd;
        }
        table {
             border-spacing: 10px;
        }
        th { text-align: left;}
        .lt {
          color: inherit;
          text-decoration: inherit;
        }
        .l {
          color: #050;
        }
        .s {
            display: inline-block;
        }
        .c {
            display: inline-block;
        }
        .e {
            /*background-color: white; Changing background color */
            border-radius: 20px; /* Making border radius */
            widt